为了开始我们的工作，首先我们需要存储和处理数据。通常，有两个重要的步骤：一，获取数据；二，得到数据后处理数据。如果不能存储数据，那么获取数据将没有意义。让我们从处理合成数据开始吧！首先，我们将介绍一个n-维数组（ndarray）, 它是MXNet用来存储和转换数据的主要工具。在MXNet中，ndarray是一个类，任何示例被称为“an ndarray”。

如果您使用过NumPy（Python中使用最广泛的科学计算包），那么您将对此章节非常熟悉。根据我们的设计，NumPy中的ndarray在MXNet中的到了提升和扩展。首先，MXNet的ndarray在CPU, GPU以及分离式云端结构中均支持异步计算，而Numpy只支持CPU。其次，MXNet的ndarray支持自动微分。这项特性使得MXNet非常适合深度学习。在本书中，当提及ndarray，除非特别声明，都意味着是MXNet中的ndarray。

**2.1 准备开始**

在此小节，我们将带您上手，运行代码，为你准备好基础数学以及算数计算工具，使您通过本书步入学习正轨。不用为意会一些数学概念或者库的功能而烦恼。接下来的小结将会回顾在实际范例背景下的素材。另外，如果您已经具备相关背景，想要深入数学概念，请跳过本结。

首先，我们要通过api，从Deep Java Library中将mxnet-engine和mxnet-native-auto模块引入到maven中。此处的api模块包括所有高阶JAVA API，它们将被用在数据处理、训练、推理当中。其中，mxnet-engine包括使用Apche MXNet框架的高阶API实现。而mxnet-native-auto模块则包含MXNet自带库中的基础运算操作符和基于C++的函数。

In [2]:


%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.9.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26

// See https://github.com/awslabs/djl/blob/master/mxnet/mxnet-engine/README.md
// for more MXNet library selection options
%maven ai.djl.mxnet:mxnet-engine:0.9.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-b



In [3]:
import ai.djl.Device;
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.index.*;

一个ndarray代表一个（可能是多维）含有数值的数组。一个有且只有一轴的ndarray在数学中对应一个向量，那么具有两轴的ndarray则对应一个矩阵。含有多于两轴但没有具体数学命名的数组，我们称之为*张量*。

新手上路，我们可以用arange创建一个行向量$\vec{x}$，包含从0开始连续的12个整数。此处默认数据类型是浮点型。每一个数值都是一个ndarray，也是ndarray的*成员element*。例如，ndarray $\vec{x}$中现在有12个*成员*。除非特殊声明，一个新的ndarray将被存储在主内存中，并且将进行基于CPU的相关计算。

In [4]:
NDManager manager = NDManager.newBaseManager();
var x = manager.arange(12);
x

ND: (12) gpu(0) int32
[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]


这里我们将使用[*NDManager*](https://javadoc.io/doc/ai.djl/api/latest/ai/djl/ndarray/NDManager.html)来创建ndarray$\vec{x}$。*NDManager*执行界面[AutoClosable](https://docs.oracle.com/javase/8/docs/api/java/lang/AutoCloseable.html)并管理由它创建的ndarray的声明周期。因为Java Garbage Collector无法监管本地内存消耗，我们需要*NDManager*的帮助。通常我们会把NDManager封装在try blocks中，这样所有的ndarray可以被及时关掉。想要了解更多关于内存管理，请阅读DJL的[相关文档](https://github.com/awslabs/djl/blob/master/docs/development/memory_management.md)。

In [5]:
try(NDManager manager = NDManager.newBaseManager()){
    NDArray x = manager.arange(12);
}

我们可以通过查看**shape**属性，获得ndarray的*shape维度*信息（每个轴的长度）。

In [6]:
x.getShape()

(12)

想要改变ndarray的维度并且不改变每个元素的数值，我们可以引用**reshape**功能。例如，我们可以这样转换我们的ndarray$\vec{x}$，从维度(1, 12)的行向量转化成维度为(3, 4)的矩阵。这是一个新的ndarray，包含相同的数值但是是由3行和4列写成的。尽管*shape维度*改变了，但是$\vec{x}$的成员没有改变。请注意，*size*不会因*reshape*而改变。

In [7]:
x = x.reshape(3, 4);
x

ND: (3, 4) gpu(0) int32
[[ 0,  1,  2,  3],
 [ 4,  5,  6,  7],
 [ 8,  9, 10, 11],
]


手动明确每一个维度使用*reshape*，是非常繁琐的过程。如果我们的目标维度是一个带有具体*shape形状*矩阵，例如当我们已知的形状是用宽度来标记，那么高度信息就是隐含的已知条件。那么为什么我们要在此时再做一次除法运算？在上述的例子中，为了得到一个3行的矩阵，我们同时明确了该矩阵应该有3行和4列。现在，当已知其它的维度信息，ndarray可以自动计算。我们通过使用-1代替我们想要ndarray自动计算的维度。在DJL中，不用像使用x.reshape(3, 4)这样，x.reshape(-1, 4)或者x.reshape(3, -1)可以得到一样的结果。

通过*create创建*的方法，只有*shape*会占用一些内存然后返回一个矩阵，此过程不会改变矩阵中任何数值。这是非常高效但我们也需要谨慎使用，因为矩阵的成员很有可能是任意数值，包括很大的任意数值。

In [8]:
manager.create(new Shape(3, 4))

ND: (3, 4) gpu(0) float32
[[ 1.12103877e-44,  1.26116862e-44,  1.40129846e-44,  1.54142831e-44],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
]


通常，我们希望矩阵初始化要不成员是0，要不是1或者其他常数，或者是明确维度分布的随机数。我们创建一个ndarray代表一个*tensor张量*，它的成员都是0，维度是(2, 3, 4)，我们这样做：

In [9]:
manager.zeros(new Shape(2, 3, 4))

ND: (2, 3, 4) gpu(0) float32
[[[0., 0., 0., 0.],
  [0., 0., 0., 0.],
  [0., 0., 0., 0.],
 ],
 [[0., 0., 0., 0.],
  [0., 0., 0., 0.],
  [0., 0., 0., 0.],
 ],
]


同样，我们可以创建一个成员都是1的*tensor张量*。

In [11]:
manager.ones(new Shape(2, 3, 4))

ND: (2, 3, 4) gpu(0) float32
[[[1., 1., 1., 1.],
  [1., 1., 1., 1.],
  [1., 1., 1., 1.],
 ],
 [[1., 1., 1., 1.],
  [1., 1., 1., 1.],
  [1., 1., 1., 1.],
 ],
]


我们更多希望从一些特定概率分布中的ndarrya为每一个成员采样随机样本值。例如，当我们在一个神经网络中创建arrays